In [448]:
import Base.Threads.@spawn

using NPZ

include("WGR-MCMClib/WGR.jl")
using .WGR

include("WGR-MCMClib/Alfheim.jl")
using .Alfheim

include("WGR-GAlib/Species.jl")
using .AlfheimG

import Base.flush
using Distributions

using Formatting
using Test
using Random

# buf = WGR.BlockBuffer(32 * Threads.nthreads())
buf = WGR.BlockBuffer(128)
setBlocks(x,y,z,b) = WGR.setBlocks!(buf, x, y, z, b)
flush() = WGR.flush!(buf)
refreshNd(a,p,f = false) = WGR.refreshNd!(buf,a,p,f)

refreshNd (generic function with 2 methods)

In [449]:
mapsize = 32
mapheight = 16
bRep = [0x00000000, 0xBA8D65FF, 0x3155A6FF, 0x96C78CFF]

4-element Vector{UInt32}:
 0x00000000
 0xba8d65ff
 0x3155a6ff
 0x96c78cff

In [450]:
###### Initialization

Nb = length(bRep)
padding = 2

_map = ones(AlfheimG.Block, mapsize + 2*padding, mapheight + 2*padding, mapsize + 2*padding)
_map[1+padding:mapsize+padding, 1:padding, 1+padding:mapsize+padding] .= 0x02

############# Create Paths energy

# Costs
costs = zeros(Int, Nb, Nb, Nb, Nb, 6)

costs[1:Nb, 1:Nb, 1, 1, 1:6] .= 10 # -> Void
costs[1:Nb, 1:Nb, 2:Nb, 1:Nb, 1:6] .= 10 # -> Solid

costs[1:Nb, 1:Nb, 1, 2:Nb, 1:2] .= 10 # -> Road, vertical (should not appear)
costs[1:Nb, 1:Nb, 1, 2:Nb, 3:6] .= 2 # -> Road, horizontal

costs[1, 2:Nb, 1, 1, 1:6] .= 3 # Road -> Void (stairs)
costs[1, 1, 1, 2:Nb, 1:6] .= 3 # Void -> Road (stairs)

# Test cost
# costs = zeros(Int, Nb, Nb, 6)
# costs[1:Nb, 1:Nb, 1:6] .= 1

@show findall(x -> x>0, costs .== 0)

# Arrays
dist = fill(Int(2^30), size(_map))
dirc = ones(UInt8, size(_map)...)

# Entrance
entranceSize = mapsize ÷ 4
entranceStart = (mapsize + 2 * padding - entranceSize) ÷ 2 + 1
entranceEnd = entranceStart + entranceSize

# 4 Entries
starts = []

dist[entranceStart:entranceEnd, 1:1 + padding, 1:padding] .= 0
starts = [starts; [[x, 1 + padding, padding] for x in entranceStart:entranceEnd]]
_map[entranceStart:entranceEnd, 1:padding, 1:padding] .= 3

dist[1:padding, 1:1 + padding, entranceStart:entranceEnd] .= 0
starts = [starts; [[padding, 1 + padding, x] for x in entranceStart:entranceEnd]]
_map[1:padding, 1:padding, entranceStart:entranceEnd] .= 3

dist[entranceStart:entranceEnd, 1:1 + padding, end:-1:end-padding+1] .= 0
starts = [starts; [[x, 1 + padding, size(_map)[3]-padding+1] for x in entranceStart:entranceEnd]]
_map[entranceStart:entranceEnd, 1:padding, end:-1:end-padding+1] .= 3

dist[end:-1:end-padding+1, 1:1 + padding, entranceStart:entranceEnd] .= 0
starts = [starts; [[size(_map)[1]-padding+1, 1 + padding, x] for x in entranceStart:entranceEnd]]
_map[end:-1:end-padding+1, 1:padding, entranceStart:entranceEnd] .= 3

@show [1, 1, 1] .+ padding

########### Create Map

testmap = AlfheimG.Map(
    _map,
    [1 + padding, 1 + padding, 1 + padding],
    [mapsize + padding, mapheight + padding, mapsize + padding]
)

########### Create objectives

pathsEnergy = Alfheim.Paths(costs, starts, [[0,0,0], [0,-1,0]], testmap.min, testmap.max, dist, dirc)
pathsWeight = 0.07

blockMarginalsE = Alfheim.BlockPropotion(0.7, AlfheimG.AirBlock) # Target: 65% Air remains
blockMarginalsW = mapsize^2 * mapheight * 6

# Energies
Etotal = Alfheim.VoxelGibbsDistribution(
    [
        blockMarginalsE,
        pathsEnergy
    ],
    [
        blockMarginalsW,
        pathsWeight
    ]
)

findall((x->begin
            #= In[450]:27 =#
            x > 0
        end), costs .== 0) = CartesianIndex{5}[]
[1, 1, 1] .+ padding = [3, 3, 3]


Main.Alfheim.VoxelGibbsDistribution(Main.Alfheim.AbstractEnergy[Main.Alfheim.BlockPropotion(0.7, 0x01, 0, Main.Alfheim.L2, Main.Alfheim.BlockPropotionState(0, 0.0), Main.Alfheim.BlockPropotionState[]), Main.Alfheim.Paths([10 3 3 3; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;;; 3 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;;; 3 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;;; 3 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10; 10 10 10 10; 10 10 10 10;;; 10 10 10 10; 10 10 10 10;

In [451]:
# g = [2, 0, 5, 5, 1, 0, 4, 5, 1, 3, 5, 5, 2, 137, 8, 10]
g = Base.rand(UInt8, 16384)

city = AlfheimG.TestCity(g)

# refreshNd(bRep[testmap.map], [0, 64, 0], true)

Main.AlfheimG.TestCity{UInt8}(UInt8[0x0f, 0xe5, 0x16, 0xc9, 0x29, 0xf4, 0x1e, 0x65, 0x73, 0xe9  …  0xa6, 0xe7, 0x19, 0xf4, 0xd5, 0x6e, 0x7a, 0xbd, 0x33, 0x9a])

In [452]:
using StatProfilerHTML

# Energy calculation
function fit(instance::AlfheimG.Specie, inmap::AlfheimG.Map, E::Alfheim.VoxelGibbsDistribution)
    copiedmap = deepcopy(inmap)
    AlfheimG.populate!(instance, copiedmap)
    dummyState = Alfheim.createMCMCState(copiedmap.map, deepcopy(E), Alfheim.DummyWalker(copiedmap.min, copiedmap.max))
    # refreshNd(bRep[copiedmap.map], [0, 64, 0], true)
    return - dummyState.p(dummyState)
end

@time fit(city, testmap, Etotal)

  3.241384 seconds (12.20 M allocations: 459.593 MiB, 14.65% gc time, 57.27% compilation time)


-110992.95592592593

In [454]:
using Base.Threads
using StatsBase

# Simple GA
nPool = Threads.nthreads() * 2
nParents = convert(Int64, nPool * 0.25)

collectedParentIdxs = collect(1:nParents)

geneLenth = 4096
pool = [AlfheimG.TestCity(Base.rand(UInt8, geneLenth)) for i in 1:nPool]
fitness = zeros(nPool)

totalIters = 1000
printInterval = 3
visInterval = 6

for iter in 1:(totalIters+1)
    
    @threads for ix in 1:nPool
        fitness[ix] = fit(pool[ix], testmap, Etotal)
    end
    
    # Choose top parents
    order = sortperm(fitness)
    parentsIdx = order[end-nParents+1:end]
    
    # Choose parents
    s1 = pool[parentsIdx[StatsBase.sample(collectedParentIdxs, nPool)]]
    s2 = pool[parentsIdx[StatsBase.sample(collectedParentIdxs, nPool)]]
    
    # Create offsprings
    pool = [AlfheimG.offspring(s1[i], s2[i], 0.01) for i in 1:nPool]
    AlfheimG.mutate!.(pool, 0.02)
    
    if iter % printInterval == 1
        printfmt("Iter {:4d} - fit: {:12.3f} ± {:<12.3f} ", iter, mean(fitness), std(fitness))
        flush(stdout)
        
        if iter % visInterval == 1
            print("(Visualizing)")
            flush(stdout)
            row = convert(Int64, floor(sqrt(nPool)))
            for ix in 1:length(pool)
                copiedmap = deepcopy(testmap)
                AlfheimG.populate!(pool[ix], copiedmap)
                refreshNd(
                    bRep[copiedmap.map], 
                    [
                           ((ix-1) % row) * (mapsize + 2*padding + 2), 
                        65, 
                        div((ix-1),  row) * (mapsize + 2*padding + 2)
                    ], 
                    true
                )
            end
        end
        
        print("\n")
    end
end

Iter    1 - fit:  -111574.808 ± 2241.301     (Visualizing)
Iter    4 - fit:  -107944.822 ± 1595.589     
Iter    7 - fit:  -107367.139 ± 1920.738     (Visualizing)
Iter   10 - fit:  -106807.621 ± 1881.282     
Iter   13 - fit:  -106462.845 ± 1493.549     (Visualizing)
Iter   16 - fit:  -104974.349 ± 1061.298     
Iter   19 - fit:  -105295.388 ± 1996.534     (Visualizing)
Iter   22 - fit:  -104685.405 ± 1642.276     
Iter   25 - fit:  -104344.174 ± 1112.378     (Visualizing)
Iter   28 - fit:  -103863.869 ± 1203.271     
Iter   31 - fit:  -103972.612 ± 1188.765     (Visualizing)
Iter   34 - fit:  -104290.063 ± 1984.511     
Iter   37 - fit:  -103733.463 ± 1457.110     (Visualizing)
Iter   40 - fit:  -103700.896 ± 2145.566     
Iter   43 - fit:  -103127.198 ± 1987.813     (Visualizing)
Iter   46 - fit:  -102868.855 ± 1388.586     
Iter   49 - fit:  -102802.318 ± 1315.389     (Visualizing)
Iter   52 - fit:  -102905.914 ± 1299.634     
Iter   55 - fit:  -102490.705 ± 1558.243     (Visualizi

LoadError: InterruptException: